In [1]:
import numpy as np
import itertools

Create an object (it's an element in the program):

In [2]:
'''
P0. Implement a list
'''

class LNode:
    def __init__(self,val):
        self.val = val
        self.next = None

In [3]:
# Create a function for decomposing the parameter mode

def parameter_modes(code):
    
    # Add leading zeros to make 5 digits
    c_str = '0'*(5-len(str(code)))+str(code)
    
    return [c_str[0],c_str[1],c_str[2], c_str[3:]]

In [4]:
#demo
icom_demo = ['3','15','3','16','1002','16','10','16','1','16','15','15','4','15','99','0','0']

Create a funciton to run a program:

In [4]:

incom = np.fromfile('07_input.txt',sep=",").astype(int)

for i, value in enumerate(incom):
    vars()['node'+str(i)]=LNode(value)

In [5]:
len(incom)

499

In [27]:
def intcode_call(signal_input, phase, incom, reinitialize = True):
    # Re-read input to reset intcom program
    if reinitialize == True:
        incom = np.fromfile('07_input.txt',sep=",").astype(int)
        
    for i, value in enumerate(incom):
        vars()['node'+str(i)]=LNode(value)
    
    mem = [phase,signal_input]



    # signal_input
    signal_in = signal_input # the puzzle suggests that this initial input should be 5 for "thermal radiator controller"

    output=[] # I think I need to track the outputs to check the result

    output_last = 0 # flag to indicate that last instruction was an output

    i = 0

    while i <= 499:
        
        #print(i)

        # decode the parameter modes
        p_modes = parameter_modes(vars()['node'+str(i)].val)

        # build instruction

        # retrieve parameters for specific opcode
        if p_modes[-1] in ['99']:
            instr = [99]
            if output_last == 0:
                print('Opcode 99 executed.')
                print('Hmmm... last instruction was not an output!')
            break

        elif p_modes[-1] in ['01', '02', '07', '08']:
            instr = [int(p_modes[-1])]
            for j in range(i+1, i+4):
                instr += [vars()['node'+str(j)].val]
            i += 4

        elif p_modes[-1] in ['03', '04']:
            instr = [int(p_modes[-1])] + [vars()['node'+str(i+1)].val]
            i += 2

        elif p_modes[-1] in ['05', '06']:
            instr = [int(p_modes[-1])]
            for j in range(i+1, i+3):
                instr += [vars()['node'+str(j)].val]
            i += 3

        else:
            print('Uh oh... intruction not understood')
            break


        # Execute instructions

        if instr[0] == 1: # addition
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            ab = a+b # calculate the overwrite value

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[3])].val = ab


        elif instr[0] == 2: # multiplication
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            ab = a*b # calculate the overwrite value

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[3])].val = ab        

        elif instr[0] == 3: # input
            output_last = 0
            # write input (mem)
            if p_modes[2] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                #print(len(mem))
                vars()['node'+str(instr[1])].val = mem[0]
                mem = mem[1:]

        elif instr[0] == 4: # output
            output_last = 1
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # write to output
            output += [a]

        elif instr[0] == 5: # jump-if-true
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

             # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                if a != 0:
                    i = b

        elif instr[0] == 6: # jump-if-false
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

             # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                if a == 0:
                    i = b

        elif instr[0] == 7: # less than
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            # apply test
            if a < b:
                ab = 1
            else:
                ab = 0                

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[3])].val = ab 


        elif instr[0] == 8: # equals
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            # apply test
            if a == b:
                ab = 1
            else:
                ab = 0                

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[3])].val = ab     

        else:
            print('Instruction not understood!')
            break

    if i > 499:
        print('End of input code reached.')
        
    # Repackage the nodes as incom
    incom = []
    for i in range(499):
        incom = incom + [vars()['node'+str(i)].val]

    return output[0], incom

In [39]:
# Generate a list of phase permutations:
perms = [i for i in itertools.permutations([0,1,2,3,4])]

In [40]:
results = []

# Accumulate results for each permutation
for perm in perms:
    # Cascade the inputs+outputs through the intcode amplifiers with the permutations of phases
    interim = intcode_call(0,perm[0], incom)[0]
    for i in range(1,5):
        interim = intcode_call(interim,perm[i], incom)[0]
        
    results.append([perm,interim])
    
results = np.array(results)

# Show highest output
np.max(results[:,1])   

11828

Part two!

I have to create a new function that doesn't re-read the input every time!

In [34]:
def intcode_call_2(signal_input, incom): #, phase):
    
    # I do still have to create nodes!
    for i, value in enumerate(incom):
        vars()['node'+str(i)]=LNode(value)
    
    mem = [signal_input]

    # signal_input
    signal_in = signal_input
    
    output=[] # I think I need to track the outputs to check the result

    output_last = 0 # flag to indicate that last instruction was an output

    i = 0

    while i <= 499:

        # decode the parameter modes
        p_modes = parameter_modes(vars()['node'+str(i)].val)

        # build instruction

        # retrieve parameters for specific opcode
        if p_modes[-1] in ['99']:
            instr = [99]
            if output_last == 0:
                print('Opcode 99 executed.')
                print('Hmmm... last instruction was not an output!')
            break

        elif p_modes[-1] in ['01', '02', '07', '08']:
            instr = [int(p_modes[-1])]
            for j in range(i+1, i+4):
                instr += [vars()['node'+str(j)].val]
            i += 4

        elif p_modes[-1] in ['03', '04']:
            instr = [int(p_modes[-1])] + [vars()['node'+str(i+1)].val]
            i += 2

        elif p_modes[-1] in ['05', '06']:
            instr = [int(p_modes[-1])]
            for j in range(i+1, i+3):
                instr += [vars()['node'+str(j)].val]
            i += 3

        else:
            print('Uh oh... intruction not understood')
            break


        # Execute instructions

        if instr[0] == 1: # addition
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            ab = a+b # calculate the overwrite value

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[3])].val = ab


        elif instr[0] == 2: # multiplication
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            ab = a*b # calculate the overwrite value

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[3])].val = ab        

        elif instr[0] == 3: # input
            output_last = 0
            # write input (mem)
            if p_modes[2] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[1])].val = mem[0]
                mem = mem[1:]

        elif instr[0] == 4: # output
            output_last = 1
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # write to output
            output += [a]

        elif instr[0] == 5: # jump-if-true
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

             # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                if a != 0:
                    i = b

        elif instr[0] == 6: # jump-if-false
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

             # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                if a == 0:
                    i = b

        elif instr[0] == 7: # less than
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            # apply test
            if a < b:
                ab = 1
            else:
                ab = 0                

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[3])].val = ab 


        elif instr[0] == 8: # equals
            output_last = 0
            # define a
            a = instr[1] # default is immediate mode
            if p_modes[2] == '0': # if parameter is position mode
                a = vars()['node'+str(instr[1])].val

            # define b
            b = instr[2] # default is immediate mode
            if p_modes[1] == '0': # if parameter is position mode
                b = vars()['node'+str(instr[2])].val

            # apply test
            if a == b:
                ab = 1
            else:
                ab = 0                

            # write         
            if p_modes[0] == '1': # I think the problem states this will never be the case...
                print('uh oh... trying to write in immediate mode???')
                break
            else:
                vars()['node'+str(instr[3])].val = ab     

        else:
            print('Instruction not understood!')
            break
            
    # Repackage the nodes as incom
    incom = []
    for i in range(499):
        incom = incom + [vars()['node'+str(i)].val]

    if i > 499:
        print('End of input code reached.')

    return output[0], incom

In [35]:
# Generate a list of phase permutations:
perms = [i for i in itertools.permutations([5,6,7,8,9])]

In [43]:
# Do we need each Amp to maintain its own ?

# Re-read input to reset intcom program
incom = np.fromfile('07_input.txt',sep=",").astype(int)

#for i, value in enumerate(incom):
#        vars()['node'+str(i)]=LNode(value)

for perm in perms[:1]:
    
    # Run first loop:
    interim, incom0 = intcode_call(signal_input =0, phase=perm[0], incom=incom, reinitialize = False)
    print(interim)
    for i in range(1,5):
        interim, vars()['incom'+str(i)] = intcode_call(signal_input=interim, phase=perm[i], incom=incom0, reinitialize = False)
        print(interim)

            
    # Now let 'er run:

    active_amp = 0
    amp_out = [0]
    last_amp_outs = np.zeros((5,1))  

    while amp_out != []: # while the amplifier is producing an output
        
        # run through the active amp
        interim, vars()['incom'+str(i)] = intcode_call_2(interim, vars()['incom'+str(i)])
        
        # place input into last amp outs:
        last_amp_outs[active_amp] = interim
        
        # Set the variable to be checked for 
        amp_out = [interim]

        # set new active amp:
        if active_amp == 4:
            active_amp = 0
        else:
            active_amp += 1
            
    # Retrieve and print last output from amp E:
    print(last_amp_out[5])

8
16
204
2458
4924


KeyError: 'node4934'

In [24]:
node102.val

3

In [16]:
parameter_modes(node0.val)

['0', '0', '0', '03']

In [28]:
# Re-read input to reset intcom program
incom = np.fromfile('07_input.txt',sep=",").astype(int)

#for i, value in enumerate(incom):
#        vars()['node'+str(i)]=LNode(value)

for perm in perms[:1]:
    
    # Run first loop:
    interim, incom = intcode_call(signal_input =0, phase=perm[0], incom=incom, reinitialize = False)
    for i in range(1,5):
        interim, incom = intcode_call(signal_input=interim, phase=perm[i], incom=incom, reinitialize = False)

In [19]:
perm[0]

5

In [13]:
a = 3
def test(a,b):
    a = 2*a
    return b

In [14]:
print(test(a,4))

print(a)



4
3


In [23]:
nodeAAA = LNode(66)

In [24]:
nodeAAA.val

66

In [25]:
nodeAAA = LNode(33)

nodeAAA.val

33

In [21]:
node1.val

8

In [25]:
test = []
for i in range(499):
        test = test + [vars()['node'+str(i)].val]